In [1]:
import numpy as np
import matplotlib.pyplot as plt
from seaborn import set_palette
import pickle
flatui = ["#3498db", "#9b59b6", "#95a5a6", "#e74c3c", "#34495e", "#2ecc71"]
set_palette(flatui)
%matplotlib inline
%config InlineBackend.figure_format = "retina"
plt.rcParams['axes.spines.top'] = False
plt.rcParams['axes.spines.right'] = False
plt.rcParams['axes.grid'] = True
plt.rcParams['grid.color'] = 'gray'
plt.rcParams['grid.linewidth'] = 0.25
plt.rcParams['grid.alpha'] = 0.2
plt.style.use('seaborn-talk')
cmap = "RdBu_r"

from scipy.signal import stft, istft, get_window
from scipy.fftpack import fft, fftshift, fftfreq
from IPython.display import Audio
from tqdm import tnrange, tqdm_notebook
from dlbeamformer_utilities import compute_steering_vectors_single_frequency,\
    compute_steering_vectors, simulate_multichannel_tf, compute_sinr,\
    compute_mvdr_tf_beamformers, check_distortless_constraint,\
    compute_mvndr_tf_beamformers, compute_lcmv_tf_beamformers,\
    compute_null_controlling_tf_beamformers,\
    compute_steering_vectors_circular
from dlbeamformers import BaseDLBeamformer, DLBeamformer
from dictionary_learning_beamformers import DictionaryLearningBeamformer
random_seed = 0


In [2]:
from configparser import ConfigParser
config = ConfigParser()
config.read('config.INI');
params = config['PARAMS']
sampling_frequency = int(params['sampling_frequency'])
n_samples_per_frame = int(params['n_samples_per_frame'])
n_fft_bins = (int) (n_samples_per_frame / 2) 
hop_size = (int) (n_samples_per_frame / 2)
stft_window_name = params['stft_window_name']
stft_window = get_window("hann", n_samples_per_frame)
stft_params = {
    "n_samples_per_frame": n_samples_per_frame,
    "n_fft_bins": n_fft_bins,
    "hop_size": hop_size,
    "window": stft_window
}
max_f_index = 128

sound_speed = int(config["CONSTANTS"]["SOUND_SPEED"])
signal_max_freq = sampling_frequency / 2
min_wavelength = sound_speed / signal_max_freq

# Microphone positions
mic_geometry_mm = [(-35.0,  20.0),
                   (-35.0, -20.0),
                   (  0.0, -40.0),
                   ( 35.0, -20.0),
                   ( 35.0,  20.0),
                   (  0.0,  40.0),
                   (  0.0,   0.0)]
pos_x = np.array([-35.0, -35.0, 0.0, 35.0, 35.0, 0.0, 0.0]) * 1e-3
n_mics = len(pos_x)
pos_y = np.array([20.0, -20.0, -40.0, -20.0, 20.0, 40.0, 0.0]) * 1e-3
pos_z = np.zeros(n_mics)
array_geometry = np.row_stack((pos_x, pos_y, pos_z))
array_geometry.shape

(3, 7)

In [35]:
theta_grid = np.array([-90])
phi_grid = np.arange(0, 360, 5)
f = 10 / n_fft_bins * (sampling_frequency/2)
print(f)
compute_steering_vectors_single_frequency(array_geometry, f, theta_grid, phi_grid)
v = compute_steering_vectors(array_geometry, sampling_frequency, n_fft_bins, theta_grid, phi_grid)

312.5


In [39]:
azimuth_grid = np.arange(0, 360, 5) # [degree]
e = np.load("./e.npy")
tau = np.load("./tau.npy")

In [40]:
e_ = compute_steering_vectors_circular(array_geometry, sampling_frequency, stft_params, azimuth_grid)

In [41]:
print(e_[2, 0, :])
print(e[2, 0, :])
i = 2
assert np.allclose(e_, e)

[0.999183-4.0413897e-02j 0.999183-4.0413897e-02j 1.      +2.8289277e-18j
 0.999183+4.0413897e-02j 0.999183+4.0413897e-02j 1.      +2.8289277e-18j
 1.      +0.0000000e+00j]
[0.999183-4.0413897e-02j 0.999183-4.0413897e-02j 1.      +2.8289277e-18j
 0.999183+4.0413897e-02j 0.999183+4.0413897e-02j 1.      +2.8289277e-18j
 1.      +0.0000000e+00j]


In [46]:
assert np.allclose(v[:, 0, :, :], e_)

In [48]:
np.linalg.norm(v[:, 0, :, :] - e_)
np.allclose(v[:, 0, :, :],  e_)

True

In [33]:
v[1]

array([[[0.99979573-0.02021108j, 0.99979573-0.02021108j,
         1.        +0.j        , 0.99979573+0.02021108j,
         0.99979573+0.02021108j, 1.        +0.j        ,
         1.        +0.j        ],
        [0.9998171 -0.01912772j, 0.9997765 -0.02114061j,
         0.999998  -0.00201329j, 0.9998171 +0.01912772j,
         0.9997765 +0.02114061j, 0.999998  +0.00201329j,
         1.        +0.j        ]]], dtype=complex64)